Digimon TCG is a relatively new TCG game, released only a short 2 years ago in both the west and the east.
Because of the nature of the game, the diffrent languages it is written in and the diffrent time frames new sets and card are released worldwide the competetive meta of the game is diffrent between the 2 regions.
The japanese (eastren) game is always a bit ahed of the english (west) game in a couple of sets, a thing that helps the english
players get ready and understand how the game will work ahead of time.
In addition this situation give the company of the game, Bandai Namco to rethink about the balance of the game based on the Japanese meta and restricet the use of cards ahed of release in the west.

In this project I will try to see if it is possible of predict the viability of a card in the english, westren meta based on how it preformed in the eastren one.


In [74]:
import time      
import os         

import bs4
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np

import requests
import json

#amount of sets
num_of_st = 13
num_of_bt = 10
num_of_ex = 2



First we will need to get a much of data as we can about each and every card in the game.
We will do this by scraping 2 famous wiki sites about the game
https://digimoncardgame.fandom.com
Will be the main site to use as there is data in the site about every card in the game, even unreleased ones
https://wikimon.net
Will be only used to get how many cards are in each set and the full list of the promo cards that are not belonging to any set

In [75]:
#Open a site as a soup object
def get_page(webpage):
    source = requests.get(webpage).text
    soup = BeautifulSoup(source,"html.parser")
    return soup

In [76]:
#load the main 4 datapoints of each card
def load_sets_info(soup):
    
    code = list()
    name = list()
    rarity = list()
    color = list()
    
    tbl=soup.find_all('table')
    for i in range (2,4):
        for a in tbl[i].find_all('a'):
            code.append(a.get_text())
            
        for tr in tbl[i].find_all('tr'):
            td = tr.find_all('td')
            th = tr.find_all('th')
            if len(th) == 0:
                nums = [1,3,4]
            else:
                nums = [0,2,3]
            if len(td) != 0 and len(td) < 10:
                name.append(td[nums[0]].get_text().replace('\n', ''))
                color.append(td[nums[1]].get_text().replace('\n', ''))
                rarity.append(td[nums[2]].get_text().replace('\n', ''))

        
    df = pd.DataFrame({"card_code":code, "card_name":name, "card_color":color, "card_rarity":rarity})
    return df

In [77]:
#load only data of the promo cards
def load_promos_info():
    
    url = 'https://wikimon.net'
    url_promo = '/Digimon_Card_Game_Promo'
    
    code = list()
    name = list()
    rarity = list()
    color = list()
    
    soup = get_page(url+url_promo)
    tbl=soup.find_all('table')

    i = 0
    while i < len(tbl):
    
        if len(tbl[i].find_all('table')) != 0:
            i = i + 1
            
        for a in tbl[i].find_all('a'):
            if len(a.get_text()) != 0:
                if (a.get_text())[0] == 'P':
                    code.append(a.get_text())
                
        for tr in tbl[i].find_all('tr'):
            td = tr.find_all('td')
            th = tr.find_all('th')

            if len(th) != 0:
                if (th[0].get_text())[0] == 'P':
                    name.append(td[0].get_text().replace('\n', ''))
                    color.append(td[2].get_text().replace('\n', ''))
                    rarity.append('Promo')
            elif len(td) != 0:
                if (td[0].get_text())[0] == 'P':
                    name.append(td[1].get_text().replace('\n', ''))
                    color.append(td[2].get_text().replace('\n', ''))
                    rarity.append('Promo')
    
        i = i + 1

    df = pd.DataFrame({"card_code":code, "card_name":name, "card_color":color, "card_rarity":rarity})
    return df

In [78]:
#create first daraframe of the main 4 datapoints of each card
url = 'https://wikimon.net'
url_tcg = '/Digimon_Card_Game'
url_promo = '/Digimon_Card_Game_Promo'

urls = list()

soup = get_page(url+url_tcg)
tbl=soup.find_all('table')

ref = [num_of_st,num_of_bt,num_of_ex]
k = 0
count = 4

for i in range (2,5):
    a = tbl[i].find_all('a')
    for j in range (0,ref[k]):
        if k == 1 and count > 0:
            urls.append(a[j*5]['href'])
            count = count - 1
        elif k == 1 and count <= 0:
            count = count - 1
            urls.append(a[j*5+count]['href'])
        else:
            urls.append(a[j*4]['href'])
    k = k + 1

df = load_sets_info(get_page(url+urls[0]))

for i in range (1, len(urls)):
    df = df.append(load_sets_info(get_page(url+urls[i])), ignore_index = True)
df = df.append(load_promos_info(), ignore_index = True)
df

,card_code,card_name,card_color,card_rarity
0,ST1-01,Koromon,Red,Uncommon
1,ST1-02,Piyomon,Red,Common
2,ST1-03,Agumon,Red,Uncommon
3,ST1-04,Dracomon,Red,Common
4,ST1-05,Birdramon,Red,Common
...,...,...,...,...
1568,P-038,Green Memory Boost!!,Green,Promo
1569,P-039,Black Memory Boost!!,Black,Promo
1570,P-040,Violet Memory Boost!!,Purple,Promo
1571,P-028,Pulsemon,Yellow,Promo


With the first dataframe out of the way we can start and scrape each and every card page to get its full datapoints

In [79]:
#get info of a single card
def get_card_info(card_url):

    url = 'https://digimoncardgame.fandom.com/wiki/'

    card_type = list()
    card_level = list()
    digimon_form = list()
    digimon_type = list()
    play_cost = list()
    dp = list()
    digimon_attribute = list()
    
    reprint_and_alt = list()
    effect = list()
    security_effect = list()
    inherited_effect = list()
    digivolve_cost1 = list()
    digivolve_cost2 = list()
    digivolve_color1 = list()
    digivolve_color2 = list()
    digivolve_lvl1 = list()
    digivolve_lvl2 = list()
    restrictionJP = list()
    restrictionW = list()
    alt_digivolution = list()

    for i in range (0, len(card_url)):
        type_flag = 0
        level_flag = 0
        form_flag = 0
        digitype_flag = 0
        play_flag = 0
        dp_flag = 0
        attr_flag = 0
        inherited_flag = 0
        first_tbl_flag = 0
        effect_flag = 0
        security_flag = 0
        digivolve_flag = 0
        digivolve1_flag = 0
        digivolve2_flag = 0
        digivolve_color1_flag = 0
        digivolve_color2_flag = 0
        digivolve_lvl1_flag = 0
        digivolve_lvl2_flag = 0
        alt_digivolution_flag = 0
        reprint_flag = 0
        restriction_flag = 0
    
        soup = get_page(url+card_url[i])
        
        for tbl in soup.find_all('table'):
            
            for tr in tbl.find_all('tr'):
                
                if digivolve_flag == 0 and 'Digivolution Requirements' in tr.get_text():
                    digivolve_flag = 1
                if digivolve_flag == 2 and 'Digivolution Requirements' in tr.get_text():
                    digivolve_flag = 3
                if restriction_flag == 0 and 'Card Restriction' in tr.get_text():
                    restriction_flag = 1
                    
                if alt_digivolution_flag == 1:
                    text = tr.get_text().replace('\n', '')
                    alt_digivolution.append(text)
                    alt_digivolution_flag = 2
                if alt_digivolution_flag != 2 and 'Alt. Digivolution Requirements' in tr.get_text():
                    alt_digivolution_flag = 1
                
                if inherited_flag == 1:
                    text = tr.get_text().replace('\n', '')
                    inherited_effect.append(text)
                    inherited_flag = 2
                if inherited_flag != 2 and 'Inherited Effect' in tr.get_text():
                    inherited_flag = 1
                    
                if effect_flag == 1:
                    text = tr.get_text().replace('\n', '')
                    effect.append(text)
                    effect_flag = 2
                if effect_flag != 2 and 'Card Effect(s)' in tr.get_text():
                    effect_flag = 1
                    
                if security_flag == 1:
                    text = tr.get_text().replace('\n', '')
                    security_effect.append(text)
                    security_flag = 2
                if security_flag != 2 and 'Security Effect' in tr.get_text():
                    security_flag = 1
                    
                for td in tr.find_all('td'):
                    
                    if restriction_flag == 1:
                        a = td.find('a')
                        text = a.get_text().replace('\n', '')
                        restrictionW.append(text)
                        
                    if restriction_flag == 2:
                        a = td.find('a')
                        text = a.get_text().replace('\n', '')
                        restrictionJP.append(text)

                    
                    if digivolve_flag == 1:
                        if digivolve1_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_cost1.append(text)
                            digivolve1_flag = 2
                        if digivolve1_flag != 2 and 'Digivolve Cost' in td.get_text():
                            digivolve1_flag = 1
                            
                        if digivolve_color1_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_color1.append(text)
                            digivolve_color1_flag = 2
                        if digivolve_color1_flag != 2 and 'Colour' in td.get_text():
                            digivolve_color1_flag = 1
                        
                        if digivolve_lvl1_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_lvl1.append(text)
                            digivolve_lvl1_flag = 2
                        if digivolve_lvl1_flag != 2 and 'Level' in td.get_text():
                            digivolve_lvl1_flag = 1
                            
                    if digivolve_flag == 3:     
                        if digivolve2_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_cost2.append(text)
                            digivolve2_flag = 2
                        if digivolve2_flag != 2 and 'Digivolve Cost' in td.get_text():
                            digivolve2_flag = 1
                            
                        if digivolve_color2_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_color2.append(text)
                            digivolve_color2_flag = 2
                        if digivolve_color2_flag != 2 and 'Colour' in td.get_text():
                            digivolve_color2_flag = 1
                        
                        if digivolve_lvl2_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digivolve_lvl2.append(text)
                            digivolve_lvl2_flag = 2
                        if digivolve_lvl2_flag != 2 and 'Level' in td.get_text():
                            digivolve_lvl2_flag = 1

                    if first_tbl_flag == 0:
                        if 'Illustrator' in td.get_text():
                            reprint_flag =reprint_flag + 1
                        
                        if attr_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digimon_attribute.append(text)
                            attr_flag = 2
                        if attr_flag != 2 and 'Attribute' in td.get_text():
                            attr_flag = 1
                    
                        if dp_flag == 1:
                            text = td.get_text().replace(' DP\n', '')
                            dp.append(text)
                            dp_flag = 2
                        if dp_flag != 2 and 'DP' in td.get_text():
                            dp_flag = 1
                
                        if play_flag == 1:
                            text = td.get_text().replace('\n', '')
                            play_cost.append(text)
                            play_flag = 2
                        if play_flag != 2 and ('Play Cost' in td.get_text() or 'Use Cost' in td.get_text()):
                            play_flag = 1
                
                        if digitype_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digimon_type.append(text)
                            digitype_flag = 2
                        if digitype_flag != 2 and 'Type' in td.get_text() and 'Card Type' not in td.get_text():
                            digitype_flag = 1
                
                        if form_flag == 1:
                            text = td.get_text().replace('\n', '')
                            digimon_form.append(text)
                            form_flag = 2
                        if form_flag != 2 and 'Form' in td.get_text():
                            form_flag = 1
                            
                        if level_flag == 1:
                            text = td.get_text().replace('\n', '').replace(' \n', '')
                            card_level.append(text)
                            level_flag = 2
                        if level_flag != 2 and 'Level' in td.get_text():
                            level_flag = 1
                            
                        if type_flag == 1:
                            text = td.get_text().replace('\n', '')
                            card_type.append(text)
                            type_flag = 2
                        if type_flag != 2 and 'Card Type' in td.get_text():
                            type_flag = 1
                                                    
                    if restriction_flag == 2:               
                        restriction_flag = 3
                    if restriction_flag == 1:               
                        restriction_flag = 2
                        
            if first_tbl_flag == 0:
                first_tbl_flag = 1
                            
            if digivolve_flag == 1:               
                digivolve_flag = 2
            if digivolve_flag == 3:               
                digivolve_flag = 4
                
        if reprint_flag == 0:
            reprint_and_alt.append(1)
        else:
            reprint_and_alt.append(reprint_flag)
            
        if alt_digivolution_flag ==0:
            alt_digivolution.append(None)
        if level_flag == 0:
            card_level.append(None)
        if form_flag == 0:
            digimon_form.append(None)
        if digitype_flag == 0:
            digimon_type.append(None)
        if dp_flag == 0:
            dp.append(None)
        if play_flag == 0:
            play_cost.append(None)
        if attr_flag == 0:
            digimon_attribute.append(None)
        if inherited_flag == 0:
            inherited_effect.append(None)
        if effect_flag == 0:
            effect.append(None)
        if security_flag == 0:
            security_effect.append(None)
        if digivolve1_flag == 0:
            digivolve_cost1.append(None)
        if digivolve2_flag == 0:
            digivolve_cost2.append(None)
        if digivolve_lvl1_flag == 0:
            digivolve_lvl1.append(None)
        if digivolve_lvl2_flag == 0:
            digivolve_lvl2.append(None)
        if digivolve_color1_flag == 0:
            digivolve_color1.append(None)
        if digivolve_color2_flag == 0:
            digivolve_color2.append(None)
                


    df = pd.DataFrame({'card_type':card_type, 'card_level':card_level, 'digimon_form':digimon_form, 'digimon_type':digimon_type,
                       'play_cost':play_cost, 'dp':dp, 'digimon_attribute':digimon_attribute, 'card_effect':effect,
                       'security_effect':security_effect, 'inherited_effect':inherited_effect,"digivolve_cost1":digivolve_cost1,
                       "digivolve_lvl1":digivolve_lvl1,"digivolve_color1":digivolve_color1,"digivolve_cost2":digivolve_cost2,
                       "digivolve_lvl2":digivolve_lvl2,"digivolve_color2":digivolve_color2, 'reprints_and_alts_amount':reprint_and_alt,
                      'alt_digivolution_req':alt_digivolution,'restriction_japan':restrictionJP,'restriction_west':restrictionW,
                      })
    return df

In [80]:
#create the main dataframe
card_url = df['card_code'].tolist()
df2 = get_card_info(card_url)
df3 = df.copy()

df3['card_type'] = df2['card_type']
df3['card_level'] = df2['card_level']
df3['digimon_form'] = df2['digimon_form']
df3['digimon_type'] = df2['digimon_type']
df3['play_cost'] = df2['play_cost']
df3['dp'] = df2['dp']
df3['digimon_attribute'] = df2['digimon_attribute']
df3['card_effect'] = df2['card_effect']
df3['security_effect'] = df2['security_effect']
df3['inherited_effect'] = df2['inherited_effect']
df3['digivolve_cost1'] = df2['digivolve_cost1']
df3['digivolve_lvl1'] = df2['digivolve_lvl1']
df3['digivolve_color1'] = df2['digivolve_color1']
df3['digivolve_cost2'] = df2['digivolve_cost2']
df3['digivolve_lvl2'] = df2['digivolve_lvl2']
df3['digivolve_color2'] = df2['digivolve_color2']
df3['reprints_and_alts_amount'] =df2['reprints_and_alts_amount']
df3['alt_digivolution_req'] = df2['alt_digivolution_req']
df3['restriction_japan'] = df2['restriction_japan']
df3['restriction_west'] = df2['restriction_west']

df3

,card_code,card_name,card_color,card_rarity,card_type,card_level,digimon_form,digimon_type,play_cost,dp,...,digivolve_cost1,digivolve_lvl1,digivolve_color1,digivolve_cost2,digivolve_lvl2,digivolve_color2,reprints_and_alts_amount,alt_digivolution_req,restriction_japan,restriction_west
0,ST1-01,Koromon,Red,Uncommon,Digi-Egg,2,In-Training,Lesser,None,None,...,None,None,None,None,None,None,1,None,Unrestricted,Unrestricted
1,ST1-02,Piyomon,Red,Common,Digimon,3,Rookie,Bird,2,3000,...,0,2,Red,None,None,None,4,None,Unrestricted,Unrestricted
2,ST1-03,Agumon,Red,Uncommon,Digimon,3,Rookie,Reptile,3,2000,...,0,2,Red,None,None,None,3,None,Unrestricted,Unrestricted
3,ST1-04,Dracomon,Red,Common,Digimon,3,Rookie,Dragon,3,4000,...,0,2,Red,None,None,None,1,None,Unrestricted,Unrestricted
4,ST1-05,Birdramon,Red,Common,Digimon,4,Champion,Giant Bird,4,5000,...,2,3,Red,None,None,None,1,None,Unrestricted,Unrestricted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,P-038,Green Memory Boost!!,Green,Promo,Option,None,None,None,3,None,...,None,None,None,None,None,None,2,None,Unrestricted,Unrestricted
1569,P-039,Black Memory Boost!!,Black,Promo,Option,None,None,None,3,None,...,None,None,None,None,None,None,2,None,Unrestricted,Unrestricted
1570,P-040,Violet Memory Boost!!,Purple,Promo,Option,None,None,None,3,None,...,None,None,None,None,None,None,2,None,Unrestricted,Unrestricted
1571,P-028,Pulsemon,Yellow,Promo,Digimon,3,Rookie,Beastkin,3,2000,...,0,2,Yellow,None,None,None,2,None,Unrestricted,Unrestricted


Now that we have the main dataframe we will do some tediuos work.
we will split the names of each set into the westren names and japanese ones as there are some diffrences in their contenets
In addition we will enter the list of rectricted cards and the date of their restriction

In [81]:
url1 = 'https://digimoncardgame.fandom.com/wiki/BT01-03:_Release_Special_Booster_Ver.1.0'
url2 = 'https://digimoncardgame.fandom.com/wiki/BT01-03:_Release_Special_Booster_Ver.1.5'

soup1 = get_page(url1)
tbl1=soup1.find('table')
soup2 = get_page(url2)
tbl2=soup2.find('table')

restriction_name = ['BT2-047','BT3-103','BT5-109','BT6-100','EX1-068','BT6-015','BT7-072']
restrictionJP_date = ['2021-03-01','2021-03-01','2022-02-01','2021-02-01','2021-02-01','2021-02-01','2021-02-01']
restrictionW_date = ['2021-04-01','2021-04-01','2022-02-25','2022-02-25','2022-02-25','2022-05-14','2022-05-14']

set_restrictionJP_date = list()
set_restrictionW_date = list()

set_name_jp = list()
set_name_west = list()

col_set_name_jp = list()
col_set_name_west = list()

for i in range (1, num_of_st + 1):
    set_name_jp.append('ST' + str(i))
for i in range (1, num_of_bt + 1):
    set_name_jp.append('BT' + str(i))  
for i in range (1, num_of_ex + 1):
    set_name_jp.append('EX' + str(i))
    
set_name_west = set_name_west + set_name_jp
set_name_west[13] = 'BT01-03: Release Special Booster Ver.1.0'
set_name_west[14] = 'BT01-03: Release Special Booster Ver.1.5'
set_name_west.remove('BT3')

for index, row in df3.iterrows():
    flag = 0
    for each in set_name_jp:
        if each+'-' in row[0]:
            col_set_name_jp.append(each) 
    if 'P' in row[0]:
        col_set_name_jp.append('Promo')
    for i in range (0, len(restriction_name)):
        if restriction_name[i] in row[0]:
            set_restrictionJP_date.append(restrictionJP_date[i])
            set_restrictionW_date.append(restrictionW_date[i])
            flag = 1
    if flag != 1:
        set_restrictionJP_date.append(None)
        set_restrictionW_date.append(None)    

            
df4 = pd.DataFrame({"set_name_japan":col_set_name_jp,"set_name_west":col_set_name_jp,
                    'set_restrictionJP_date':set_restrictionJP_date,'restriction_date_west':set_restrictionW_date})

df3['set_name_japan'] = df4['set_name_japan']
df3['set_name_west'] = df4['set_name_west']
df3['set_restrictionJP_date'] = df4['set_restrictionJP_date']
df3['restriction_date_west'] = df4['restriction_date_west']

for i in range (0,len(df3['card_code'])):
    for tr in tbl1.find_all('tr'):
        th = tr.find('th')
        if df3.at[i,'card_code'] in th.get_text():
            df3.at[i,'set_name_west'] = 'BT01-03: Release Special Booster Ver.1.0'
            
    for tr in tbl2.find_all('tr'):
        th = tr.find('th')
        if df3.at[i,'card_code'] in th.get_text():
            df3.at[i,'set_name_west'] = 'BT01-03: Release Special Booster Ver.1.5'
df3['set_restrictionJP_date'] = pd.to_datetime(df3['set_restrictionJP_date'], format='%Y-%m-%d')
df3['restriction_date_west'] = pd.to_datetime(df3['restriction_date_west'], format='%Y-%m-%d')
df3

,card_code,card_name,card_color,card_rarity,card_type,card_level,digimon_form,digimon_type,play_cost,dp,...,digivolve_lvl2,digivolve_color2,reprints_and_alts_amount,alt_digivolution_req,restriction_japan,restriction_west,set_name_japan,set_name_west,set_restrictionJP_date,restriction_date_west
0,ST1-01,Koromon,Red,Uncommon,Digi-Egg,2,In-Training,Lesser,None,None,...,None,None,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT
1,ST1-02,Piyomon,Red,Common,Digimon,3,Rookie,Bird,2,3000,...,None,None,4,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT
2,ST1-03,Agumon,Red,Uncommon,Digimon,3,Rookie,Reptile,3,2000,...,None,None,3,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT
3,ST1-04,Dracomon,Red,Common,Digimon,3,Rookie,Dragon,3,4000,...,None,None,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT
4,ST1-05,Birdramon,Red,Common,Digimon,4,Champion,Giant Bird,4,5000,...,None,None,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,P-038,Green Memory Boost!!,Green,Promo,Option,None,None,None,3,None,...,None,None,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT
1569,P-039,Black Memory Boost!!,Black,Promo,Option,None,None,None,3,None,...,None,None,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT
1570,P-040,Violet Memory Boost!!,Purple,Promo,Option,None,None,None,3,None,...,None,None,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT
1571,P-028,Pulsemon,Yellow,Promo,Digimon,3,Rookie,Beastkin,3,2000,...,None,None,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT


Lastly before moving to the data about the use of the cards we will add the release date of every set.
with this we will be able to cut the meta even more into chunks of each major release

In [82]:
jp_date = ['2020-04-24','2020-04-24','2020-04-24','2020-11-27','2020-11-27','2020-11-27','2021-04-03','2021-04-03',
          '2021-10-29','2021-10-29','2021-11-26','2022-04-22','2022-04-22','2020-03-15','2020-07-22','2020-10-30',
          '2020-12-18','2021-02-26','2021-05-28','2021-08-27','2021-09-26',
          '2022-02-25','2022-05-27','2021-07-30','2021-12-24']
west_date = ['2020-11-27','2020-11-27','2020-11-27','2021-11-06','2021-11-06','2021-11-06','2021-10-01','2021-10-01',
            '2022-05-13','2022-05-13','NaN','2022-10-14','2022-10-14','2020-11-27','2021-03-12','2021-06-11',
            '2021-08-06','2021-10-01', '2022-03-04', '2022-05-13','2022-07-29','2022-10-14','2021-12-10','2022-06-24']

set_jp_date = list()
set_west_date = list()

for i in range (0, len(df3['card_code'])):
    for j in range (0, len(set_name_jp)):
        if df3.at[i,'set_name_japan'] == set_name_jp[j]:
            set_jp_date.append(jp_date[j])
    if df3.at[i,'set_name_japan'] == 'Promo':
        set_jp_date.append(None)
        
    for j in range (0, len(set_name_west)):
        if df3.at[i,'set_name_west'] == set_name_west[j]:
            set_west_date.append(west_date[j])
    if df3.at[i,'set_name_west'] == 'Promo':
        set_west_date.append(None)
            
df5 = pd.DataFrame({"set_west_date":set_west_date,"set_jp_date":set_jp_date})

df3['set_west_date'] = df5['set_west_date'] 
df3['set_jp_date'] = df5['set_jp_date']

df3['set_west_date'] = pd.to_datetime(df3['set_west_date'], format='%Y-%m-%d')
df3['set_jp_date'] = pd.to_datetime(df3['set_jp_date'], format='%Y-%m-%d')

df3

,card_code,card_name,card_color,card_rarity,card_type,card_level,digimon_form,digimon_type,play_cost,dp,...,reprints_and_alts_amount,alt_digivolution_req,restriction_japan,restriction_west,set_name_japan,set_name_west,set_restrictionJP_date,restriction_date_west,set_west_date,set_jp_date
0,ST1-01,Koromon,Red,Uncommon,Digi-Egg,2,In-Training,Lesser,None,None,...,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT,2020-11-27,2020-04-24
1,ST1-02,Piyomon,Red,Common,Digimon,3,Rookie,Bird,2,3000,...,4,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT,2020-11-27,2020-04-24
2,ST1-03,Agumon,Red,Uncommon,Digimon,3,Rookie,Reptile,3,2000,...,3,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT,2020-11-27,2020-04-24
3,ST1-04,Dracomon,Red,Common,Digimon,3,Rookie,Dragon,3,4000,...,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT,2020-11-27,2020-04-24
4,ST1-05,Birdramon,Red,Common,Digimon,4,Champion,Giant Bird,4,5000,...,1,None,Unrestricted,Unrestricted,ST1,ST1,NaT,NaT,2020-11-27,2020-04-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,P-038,Green Memory Boost!!,Green,Promo,Option,None,None,None,3,None,...,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT,NaT,NaT
1569,P-039,Black Memory Boost!!,Black,Promo,Option,None,None,None,3,None,...,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT,NaT,NaT
1570,P-040,Violet Memory Boost!!,Purple,Promo,Option,None,None,None,3,None,...,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT,NaT,NaT
1571,P-028,Pulsemon,Yellow,Promo,Digimon,3,Rookie,Beastkin,3,2000,...,2,None,Unrestricted,Unrestricted,Promo,Promo,NaT,NaT,NaT,NaT


With this out of the way it is time to get the data of the diffrent metas in the game.
https://digimonmeta.com
is a site that has deck list of each winning deck that won in a tournement splited into diffrent dates, metas and regions.
We will try to split the metas even further from this site by the release dates of the major sets (BTs and EXs)

We will count how many timnes a card was showing in these decks in each meta and bu that we will determine if the card was highly playable or nott.

In the first meta we will start from BT3 in japan and 1.5 in the west to get more accurate comparation

In [83]:
def meta_info(urls):

    jp_metas_lists = []
    jp_metas_lists_sizes = list()
    west_metas_lists = []
    west_metas_lists_sizes = list()
    
    meta_jp1 = list()
    meta_jp2 = list()
    meta_jp3 = list()
    meta_jp4 = list()
    meta_jp5 = list()
    meta_jp6 = list()
    meta_jp7 = list()
    meta_jp8 = list()
    meta_jp9 = list()
    meta_jp10 = list()
    
    meta_west1 = list()
    meta_west2 = list()
    meta_west3 = list()
    meta_west4 = list()
    meta_west5 = list()
    meta_west6 = list()
    meta_west7 = list()
    meta_west8 = list()
    
    months = {'Jan':'01','Feb':'02','Mar':'03','Apr':'04',
              'May':'05','Jun':'06','Jul':'07','Aug':'08',
              'Sep':'09','Oct':'10','Nov':'11','Dec':'12'}
    
    for i in range (0, 7):
        soup = get_page(urls[i])
        tbody = soup.find('tbody')
        for tr in tbody.find_all('tr'):
            td = tr.find('td', attrs = {'class':'column-6'})
            date = td.get_text()
            for key, value in months.items():
                date = date.replace(key, value)
            date = pd.to_datetime(date, format='%m %d, %Y');
            if date >= pd.to_datetime('30/10/2020', format='%d/%m/%Y'):
                
                if date <= pd.to_datetime('17/12/2020', format='%d/%m/%Y'):
                    try:
                        meta_jp1.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('26/02/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp2.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('26/05/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp3.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('29/07/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp4.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('25/08/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp5.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('24/11/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp6.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('23/12/2021', format='%d/%m/%Y'):
                    try:
                        meta_jp7.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('26/02/2022', format='%d/%m/%Y'):
                    try:
                        meta_jp8.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('25/05/2022', format='%d/%m/%Y'):
                    try:
                        meta_jp9.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('01/07/2022', format='%d/%m/%Y'):
                    try:
                        meta_jp10.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass         

    for i in range (7, len(urls)):
        soup = get_page(urls[i])
        tbody = soup.find('tbody')
        for tr in tbody.find_all('tr'):
            td = tr.find('td', attrs = {'class':'column-6'})
            date = td.get_text()
            for key, value in months.items():
                date = date.replace(key, value)
            date = pd.to_datetime(date, format='%m %d, %Y')
            if date >= pd.to_datetime('12/03/2021', format='%d/%m/%Y'):
                
                if date <= pd.to_datetime('10/06/2021', format='%d/%m/%Y'):
                    try:
                        meta_west1.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('05/08/2021', format='%d/%m/%Y'):
                    try:
                        meta_west2.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('01/10/2021', format='%d/%m/%Y'):
                    try:
                        meta_west3.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('09/12/2021', format='%d/%m/%Y'):
                    try:
                        meta_west4.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('03/03/2022', format='%d/%m/%Y'):
                    try:
                        meta_west5.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('12/05/2022', format='%d/%m/%Y'):
                    try:
                        meta_west6.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('23/06/2022', format='%d/%m/%Y'):
                    try:
                        meta_west7.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                elif date <= pd.to_datetime('01/07/2022', format='%d/%m/%Y'):
                    try:
                        meta_west8.append(url + tr.find('a')['href'])
                    except KeyError:
                        pass
                    
    jp_metas_lists.append(meta_jp1)
    jp_metas_lists.append(meta_jp2)
    jp_metas_lists.append(meta_jp3)
    jp_metas_lists.append(meta_jp4)
    jp_metas_lists.append(meta_jp5)
    jp_metas_lists.append(meta_jp6)
    jp_metas_lists.append(meta_jp7)
    jp_metas_lists.append(meta_jp8)
    jp_metas_lists.append(meta_jp9)
    jp_metas_lists.append(meta_jp10)
    
    west_metas_lists.append(meta_west1)
    west_metas_lists.append(meta_west2)
    west_metas_lists.append(meta_west3)
    west_metas_lists.append(meta_west4)
    west_metas_lists.append(meta_west5)
    west_metas_lists.append(meta_west6)
    west_metas_lists.append(meta_west7)
    west_metas_lists.append(meta_west8)
    

    
    return jp_metas_lists, west_metas_lists

In [84]:
url = 'https://digimonmeta.com/deck-list/'
urls = list()

soup = get_page(url)
for div in soup.find_all('div', attrs = {"class":"elementor-image"}):
    for a in div.find_all('a'):
        urls.append(a['href'])
        
jp_metas_list, west_metas_list = meta_info(urls)


In [85]:
def break_to_amounts(decks):
    meta = []
    count = 0
    for deck in decks:
        flag = True
        while(flag):
            try:
                soup = get_page(deck)
                flag = False
            except requests.exceptions.ConnectionError:
                print("fail")
                time.sleep(10)
            
        div = soup.find('div', attrs = {'style':"font-family:monospace;"})
        if type(div) == type(None):
            continue
        for i in range (0, len(df3['card_code'])):
            if deck == decks[0]:
                meta.append((div.get_text()).count(df3.at[i,'card_code']))
            else:
                meta[i] = meta[i] + div.get_text().count(df3.at[i,'card_code'])
        print(count + 1)
        count = count + 1
        
    return meta

In [45]:
jp_meta_decks = []
for decks in jp_metas_list:
    jp_meta_decks.append(break_to_amounts(decks))
jp_meta_decks

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30

[[46,
  155,
  201,
  0,
  0,
  186,
  144,
  0,
  3,
  171,
  1,
  20,
  0,
  0,
  1,
  142,
  1,
  121,
  11,
  0,
  0,
  6,
  76,
  15,
  0,
  19,
  4,
  0,
  103,
  0,
  10,
  26,
  7,
  7,
  0,
  49,
  0,
  0,
  87,
  0,
  34,
  0,
  1,
  71,
  0,
  0,
  2,
  16,
  3,
  26,
  43,
  65,
  0,
  8,
  0,
  82,
  0,
  1,
  28,
  0,
  50,
  0,
  29,
  0,
  12,
  0,
  19,
  23,
  0,
  0,
  0,
  34,
  0,
  0,
  15,
  0,
  0,
  6,
  2,
  8,
  15,
  44,
  0,
  56,
  54,
  0,
  0,
  117,
  0,
  6,
  0,
  2,
  9,
  12,
  69,
  99,
  0,
  171,
  0,
  0,
  0,
  0,
  0,
  197,
  177,
  0,
  0,
  0,
  0,
  0,
  0,
  142,
  0,
  139,
  0,
  0,
  0,
  0,
  0,
  91,
  18,
  0,
  0,
  0,
  0,
  103,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  68,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  115,
  148,
  201,
  105,
  4,
  2,
  144,
  0,
  0,
  84,
  170,
  0,
  8,
  118,
  36,
  142,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [46]:
west_meta_decks = []
for decks in west_metas_list:
    west_meta_decks.append(break_to_amounts(decks))
west_meta_decks

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
11

[[7,
  21,
  32,
  0,
  0,
  32,
  28,
  0,
  0,
  41,
  2,
  10,
  0,
  0,
  0,
  61,
  0,
  40,
  0,
  0,
  0,
  0,
  24,
  2,
  0,
  3,
  0,
  2,
  45,
  0,
  21,
  23,
  1,
  6,
  0,
  46,
  0,
  0,
  49,
  0,
  19,
  4,
  0,
  30,
  0,
  0,
  0,
  13,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  26,
  0,
  0,
  0,
  0,
  0,
  32,
  33,
  0,
  0,
  0,
  0,
  0,
  0,
  61,
  0,
  34,
  0,
  0,
  0,
  0,
  0,
  41,
  4,
  0,
  0,
  0,
  0,
  45,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  25,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  27,
  32,
  20,
  0,
  0,
  28,
  0,
  0,
  19,
  30,
  0,
  4,
  48,
  8,
  61,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [86]:
df6 = pd.DataFrame({"west_meta_decks1":west_meta_decks[0], "west_meta_decks2":west_meta_decks[1],"west_meta_decks3":west_meta_decks[2],
                   "west_meta_decks4":west_meta_decks[3],"west_meta_decks5":west_meta_decks[4],"west_meta_decks6":west_meta_decks[5],
                   "west_meta_decks7":west_meta_decks[6],"west_meta_decks8":west_meta_decks[7],
                   "jp_meta_decks1":jp_meta_decks[0],"jp_meta_decks2":jp_meta_decks[1],"jp_meta_decks3":jp_meta_decks[2],
                    "jp_meta_decks4":jp_meta_decks[3],"jp_meta_decks5":jp_meta_decks[4],"jp_meta_decks6":jp_meta_decks[5],
                    "jp_meta_decks7":jp_meta_decks[6],"jp_meta_decks8":jp_meta_decks[7],"jp_meta_decks9":jp_meta_decks[8],
                    "jp_meta_decks10":jp_meta_decks[9]})

df3["west_meta_decks1"] = df6["west_meta_decks1"]
df3["west_meta_decks2"] = df6["west_meta_decks2"]
df3["west_meta_decks3"] = df6["west_meta_decks3"]
df3["west_meta_decks4"] = df6["west_meta_decks4"]
df3["west_meta_decks5"] = df6["west_meta_decks5"]
df3["west_meta_decks6"] = df6["west_meta_decks6"]
df3["west_meta_decks7"] = df6["west_meta_decks7"] 
df3["west_meta_decks8"] = df6["west_meta_decks8"]
df3["jp_meta_decks1"] = df6["jp_meta_decks1"]
df3["jp_meta_decks2"] = df6["jp_meta_decks2"]
df3["jp_meta_decks3"] = df6["jp_meta_decks3"]
df3["jp_meta_decks4"] = df6["jp_meta_decks4"]
df3["jp_meta_decks5"] = df6["jp_meta_decks5"]
df3["jp_meta_decks6"] = df6["jp_meta_decks6"]
df3["jp_meta_decks7"] = df6["jp_meta_decks7"]
df3["jp_meta_decks8"] = df6["jp_meta_decks8"]
df3["jp_meta_decks9"] = df6["jp_meta_decks9"]
df3["jp_meta_decks10"] = df6["jp_meta_decks10"]

df_clean


,card_code,card_name,card_color,card_rarity,card_type,card_level,digimon_form,digimon_type,play_cost,dp,...,jp_meta_decks1,jp_meta_decks2,jp_meta_decks3,jp_meta_decks4,jp_meta_decks5,jp_meta_decks6,jp_meta_decks7,jp_meta_decks8,jp_meta_decks9,jp_meta_decks10
0,ST1-01,Koromon,Red,Uncommon,Digi-Egg,2,In-Training,Lesser,None,None,...,46,10,51,44,2,0,0,5,4,0
1,ST1-02,Piyomon,Red,Common,Digimon,3,Rookie,Bird,2,3000,...,155,39,18,37,0,7,0,0,1,0
2,ST1-03,Agumon,Red,Uncommon,Digimon,3,Rookie,Reptile,3,2000,...,201,61,125,105,26,0,6,13,17,3
3,ST1-04,Dracomon,Red,Common,Digimon,3,Rookie,Dragon,3,4000,...,0,0,0,0,0,0,0,0,0,0
4,ST1-05,Birdramon,Red,Common,Digimon,4,Champion,Giant Bird,4,5000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,P-076,Deltamon,Black/Red,Promo,Digimon,4,Champion,Composite,5,6000,...,0,0,0,0,0,0,0,0,11,0
1532,P-077,Wizarmon,Purple,Promo,Digimon,4,Champion,Wizard,5,5000,...,0,0,0,0,0,0,0,0,13,0
1533,P-079,Agumon,Red,Promo,Digimon,3,Rookie,Reptile,3,2000,...,0,0,0,0,0,0,0,0,0,0
1534,P-080,Labramon,Purple,Promo,Digimon,3,Rookie,Beast,3,2000,...,0,0,0,0,0,0,0,0,0,0


In [87]:
west_amount = list()
for i in range (0,len(west_metas_list)):
    west_amount.append(len(west_metas_list[i]))
jp_amount = list()
for i in range (0,len(jp_metas_list)):
    jp_amount.append(len(jp_metas_list[i]))
    
west_amount.append(0)    
west_amount.append(0)    

df6 = pd.DataFrame({"west_amount":west_amount, "jp_amount":jp_amount})
df6.to_csv('file2.csv',index=False)    
df3.to_csv('file1.csv',index=False)
df3

,card_code,card_name,card_color,card_rarity,card_type,card_level,digimon_form,digimon_type,play_cost,dp,...,jp_meta_decks1,jp_meta_decks2,jp_meta_decks3,jp_meta_decks4,jp_meta_decks5,jp_meta_decks6,jp_meta_decks7,jp_meta_decks8,jp_meta_decks9,jp_meta_decks10
0,ST1-01,Koromon,Red,Uncommon,Digi-Egg,2,In-Training,Lesser,None,None,...,46,10,51,44,2,0,0,5,4,0
1,ST1-02,Piyomon,Red,Common,Digimon,3,Rookie,Bird,2,3000,...,155,39,18,37,0,7,0,0,1,0
2,ST1-03,Agumon,Red,Uncommon,Digimon,3,Rookie,Reptile,3,2000,...,201,61,125,105,26,0,6,13,17,3
3,ST1-04,Dracomon,Red,Common,Digimon,3,Rookie,Dragon,3,4000,...,0,0,0,0,0,0,0,0,0,0
4,ST1-05,Birdramon,Red,Common,Digimon,4,Champion,Giant Bird,4,5000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1568,P-038,Green Memory Boost!!,Green,Promo,Option,None,None,None,3,None,...,0,0,15,25,3,3,9,31,111,34
1569,P-039,Black Memory Boost!!,Black,Promo,Option,None,None,None,3,None,...,0,0,8,39,7,19,9,31,85,39
1570,P-040,Violet Memory Boost!!,Purple,Promo,Option,None,None,None,3,None,...,0,0,27,39,0,30,4,18,15,4
1571,P-028,Pulsemon,Yellow,Promo,Digimon,3,Rookie,Beastkin,3,2000,...,0,0,61,88,32,27,7,6,4,3


In [88]:
df6

,west_amount,jp_amount
0,61,258
1,151,210
2,151,270
3,155,288
4,142,81
5,197,313
6,117,80
7,14,251
8,0,282
9,0,118
